In [2]:
from Utils import load_results, loadBestKnown, loadProblems, arpd, average_pairwise_distance
import numpy as np

In [3]:
bk_solutions = {f"j30{i//10 + 1}_{(i%10)+1}": makespan for i, makespan in enumerate(loadBestKnown(30))}

In [25]:
def print_results(filename: str):
    results = load_results(filename=filename)
    cycles = np.array([len(result.best_history) for result in results])
    print(filename.upper())
    print("---------------------------")
    print(f"Average number of cycles: {np.average(cycles)}")
    print(f"Standar desviation: {np.std(cycles)}")
    print(f"Minimum cycles: {np.min(cycles)}")
    print(f"Maximum cycles: {np.max(cycles)}")
    print("")
    desviation = np.array([result.best.makespan - bk_solutions[result.problem_id] for result in results])
    print(f"Average number of desviation: {np.average(desviation)}")
    print(f"Standar desviation (of desviations): {np.std(desviation)}")
    print(f"Minimum desviation: {np.min(desviation)}")
    print(f"Maximum desviation: {np.max(desviation)}")
    print("---------------------------")
    print("")

In [ ]:
population_size = [14, 15, 16, 17]
limit_num = [125]
for size in population_size:
    for limit in limit_num:
        print_results(filename="Results/j30-N{size}-L{limit}.json")

In [ ]:
proportions = [0.45]
sampling_rate = [5, 10, 15]
for proportion in proportions:
    for sampling in sampling_rate:
        print_results(filename=f"Results/j30-P{int(100*proportion)}-S{sampling}.json")

In [32]:
print_results(filename="Results/j30-N17-L125-P45-S15.json")

Results successfully loaded from Results/j30-N17-L125-P45-S15.json
RESULTS/J30-N17-L125-P45-S15.JSON
---------------------------
Average number of cycles: 357.64
Standar desviation: 562.2713494390409
Minimum cycles: 4
Maximum cycles: 1472

Average number of desviation: 0.26
Standar desviation (of desviations): 0.5936328831862332
Minimum desviation: 0
Maximum desviation: 2
---------------------------



In [33]:
print_results(filename="Results/j30-random-init-abc.json")

Results successfully loaded from Results/j30-random-init-abc.json
RESULTS/J30-RANDOM-INIT-ABC.JSON
---------------------------
Average number of cycles: 545.10625
Standar desviation: 632.0100763655624
Minimum cycles: 2
Maximum cycles: 1472

Average number of desviation: 0.475
Standar desviation (of desviations): 0.9079509898667439
Minimum desviation: 0
Maximum desviation: 6
---------------------------

